In [1]:
#import libraries
import numpy 
import matplotlib.pyplot as plt
import pandas as pd
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import SimpleRNN
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from keras.callbacks import ModelCheckpoint, EarlyStopping

### Load dataset

In [2]:
#Read the file
df = pd.read_csv(r'data_mse_mse_historic_data_active.csv', sep="\t")

In [3]:
print(df.shape)
print(df.columns)

(43441, 19)
Index(['Date', 'stock_id', 'Open', 'High', 'Low', 'Close', 'Volume',
       'Adj Close', 'Quantity', 'Average', 'Change %', 'Volume Total', 'Ratio',
       'Ratio 1m', 'Ratio 3m', 'Ratio 6m', 'Ratio 1y', 'Ratio 2y', 'Ratio 3y'],
      dtype='object')


### Missing Values

In [4]:
# Find unique users
df['Date'] = pd.to_datetime(df['Date'])
users = df['stock_id'].unique()
dates = df.set_index('Date').index

In [8]:
idx = pd.MultiIndex.from_product((dates, users), names=['Date', 'stock_id'])
df1 = df.set_index(['Date', 'stock_id']).reindex(idx, fill_value=0).reset_index()

In [9]:
df1 = df.join(df.groupby('stock_id')['Date'].agg(['min', 'max']), on='stock_id')

In [10]:
#All users have different start_date so we need to dial with it
df2 = df1[df1['Date'] > df1['min']]

In [11]:
df2['Open'] = df2['Open'].replace(to_replace=0, method='ffill')
df2['Close'] = df2['Close'].replace(to_replace=0, method='ffill')
df2['High'] = df2['High'].replace(to_replace=0, method='ffill')
df2['Low'] = df2['Low'].replace(to_replace=0, method='ffill')
df2['Adj Close'] = df2['Adj Close'].replace(to_replace=0, method='ffill')
df2['Average'] = df2['Average'].replace(to_replace=0, method='ffill')

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pyd

In [13]:
# if we need to get the same value as previuos row
#df2 = df2.replace(to_replace=0, method='ffill')

In [12]:
df2


,Date,stock_id,Open,High,Low,Close,Volume,Adj Close,Quantity,Average,...,Volume Total,Ratio,Ratio 1m,Ratio 3m,Ratio 6m,Ratio 1y,Ratio 2y,Ratio 3y,min,max
0,2020-08-25,ALK,12290.0,12222.0,12200.0,12200.0,3660440,12200.0,300,12201.47,...,3660440,0.878,0.971,0.971,0.937,0.916,0.916,0.916,1997-01-09,2020-08-25
1,2020-08-24,ALK,12290.0,12290.0,12289.0,12290.0,73738,12290.0,6,12289.67,...,73738,0.885,0.978,0.978,0.944,0.922,0.922,0.922,1997-01-09,2020-08-25
2,2020-08-21,ALK,12223.0,12290.0,12289.0,12290.0,614476,12290.0,50,12289.52,...,614476,0.885,0.978,0.978,0.944,0.922,0.922,0.922,1997-01-09,2020-08-25
3,2020-08-20,ALK,12300.0,12300.0,12223.0,12223.0,651640,12223.0,53,12295.09,...,651640,0.885,0.979,0.979,0.945,0.923,0.923,0.923,1997-01-09,2020-08-25
4,2020-08-19,ALK,12300.0,12300.0,12300.0,12300.0,2952000,12300.0,240,12300.00,...,2952000,0.885,0.979,0.979,0.945,0.923,0.923,0.923,1997-01-09,2020-08-25
5,2020-08-18,ALK,12399.0,12300.0,12300.0,12300.0,1549800,12300.0,126,12300.00,...,1549800,0.885,0.979,0.979,0.945,0.923,0.923,0.923,1997-01-09,2020-08-25
6,2020-08-17,ALK,12500.0,12399.0,12399.0,12399.0,24798,12399.0,2,12399.00,...,24798,0.893,0.987,0.987,0.953,0.930,0.930,0.930,1997-01-09,2020-08-25
7,2020-08-14,ALK,12500.0,12500.0,12450.0,12500.0,799460,12500.0,64,12491.56,...,799460,0.899,0.994,0.994,0.960,0.937,0.937,0.937,1997-01-09,2020-08-25
8,2020-08-12,ALK,12500.0,12500.0,12400.0,12500.0,1049000,12500.0,84,12488.10,...,1049000,0.899,0.994,0.994,0.960,0.937,0.937,0.937,1997-01-09,2020-08-25
9,2020-08-11,ALK,12350.0,12600.0,12350.0,12500.0,2930385,12500.0,235,12469.72,...,2930385,0.898,0.992,0.992,0.958,0.936,0.936,0.936,1997-01-09,2020-08-25


In [18]:
df2.to_csv('ex.csv')